In [4]:
from classes import *
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as MplPolygon
import geopandas as gpd
from shapely.geometry import Polygon as ShapelyPolygon
import json
import folium

from os import path

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
data1_path = "/Users/sebastiangmur/Projekte/Uzh/GEO877_Spatial_Algorithms/SpatialAlgorithms/data/Polygons_no_mountains.geojson"
data2_path = "/Users/sebastiangmur/Projekte/Uzh/GEO877_Spatial_Algorithms/SpatialAlgorithms/data/Polygons_with_mountains.geojson"
data3_path = "/Users/sebastiangmur/Projekte/Uzh/GEO877_Spatial_Algorithms/SpatialAlgorithms/data/swissnames_points_json_export.geojson"

json_files = [(data1_path, "Gemeinden_ohne_Berge"),
(data2_path, "Gemeinden_mit_Berge"),
(data3_path, "Swissnames_Points")]


In [6]:
data = process_json_file(json_files)

In [7]:
data['Gemeinden_ohne_Berge']
data['Gemeinden_mit_Berge']
#data['Swissnames_Points']

[Polygon Damphreux-Lugnez consisting of 11 points,
 Polygon Herznach-Ueken consisting of 8 points,
 Polygon Menziken consisting of 8 points,
 Polygon Les Breuleux consisting of 12 points,
 Polygon Neckertal consisting of 27 points,
 Polygon Münchenbuchsee consisting of 13 points,
 Polygon Andelfingen consisting of 27 points,
 Polygon Schwende-Rüte consisting of 19 points,
 Polygon Val Mara consisting of 14 points,
 Polygon Aeugst am Albis consisting of 7 points,
 Polygon Affoltern am Albis consisting of 9 points,
 Polygon Bonstetten consisting of 8 points,
 Polygon Hausen am Albis consisting of 10 points,
 Polygon Hedingen consisting of 6 points,
 Polygon Kappel am Albis consisting of 10 points,
 Polygon Knonau consisting of 6 points,
 Polygon Maschwanden consisting of 9 points,
 Polygon Mettmenstetten consisting of 14 points,
 Polygon Obfelden consisting of 17 points,
 Polygon Ottenbach consisting of 10 points,
 Polygon Rifferswil consisting of 9 points,
 Polygon Stallikon consisting 

In [8]:
data['Swissnames_Points']

[Point Reiat (x=2696044.4240001203, y=1288550.4469925512, objektart=Zollamt 24h 24h) ,
 Point Ramsen (x=2703851.5070001264, y=1285487.2229927967, objektart=Zollamt 24h 24h) ,
 Point Kreuzlingen-Autobahn (x=2729311.4010001486, y=1279895.6999932611, objektart=Zollamt 24h 24h) ,
 Point Kreuzlingen-Autobahn (x=2729306.5150001487, y=1279890.8639932596, objektart=Zollamt 24h 24h) ,
 Point Kreuzlingen-Emmishofen (x=2730025.3360001477, y=1279823.870993266, objektart=Zollamt 24h 24h) ,
 Point Kreuzlingen Hauptzoll (x=2730307.198000148, y=1279744.7339932749, objektart=Zollamt 24h 24h) ,
 Point Romanshorn (x=2746185.7880001455, y=1269859.512994081, objektart=Zollamt 24h 24h) ,
 Point Au (x=2766261.223000134, y=1255638.4899952735, objektart=Zollamt 24h 24h) ,
 Point St. Margrethen (x=2765922.124000141, y=1258402.1639950455, objektart=Zollamt 24h 24h) ,
 Point Rheineck (x=2762607.85900014, y=1259288.3229949633, objektart=Zollamt 24h 24h) ,
 Point Montlingen (x=2762792.9040001044, y=1244816.43299617